In [32]:
import os
import lancedb
import polars as pl

#print(filenames,len(filenames))

In [34]:
#from ipywidgets import interact, interactive, fixed, interact_manual, controls
import ipywidgets
from sentence_transformers import SentenceTransformer, util, InputExample, losses
from langchain_text_splitters import RecursiveCharacterTextSplitter
model = SentenceTransformer("thenlper/gte-base")

filenames = os.listdir("../config_02")   #Not sure why .DS_Store shows up in some folders, if not then comment out the .remove line
#print(filenames)
#filenames.remove('.DS_Store')
print(filenames,len(filenames))

['vectorized_config_02_data_pt0.parquet', 'vectorized_config_02_data_pt9.parquet', 'vectorized_config_02_data_pt8.parquet', 'vectorized_config_02_data_pt1.parquet', 'vectorized_config_02_data_pt3.parquet', 'vectorized_config_02_data_pt2.parquet', 'vectorized_config_02_data_pt7.parquet', 'vectorized_config_02_data_pt10.parquet', 'vectorized_config_02_data_pt6.parquet', 'vectorized_config_02_data_pt4.parquet', 'vectorized_config_02_data_pt5.parquet'] 11


In [35]:
## Initialize a database
db = lancedb.connect("../config_02/Config_02_subreddit_metadata_lancedb")

In [37]:
%%time

## Load the first data file
data = pl.read_parquet("../config_02/vectorized_config_02_data_pt0.parquet")
## Create a table in the database based on the first file, ignore warning if the table already exists. Use mode="overwrite" if you want scorched earth
table = db.create_table("Config_02_subreddit_metadata_table", data=data,exist_ok=True)
    
## Load the remaining data into the table, one file at a time, there are 11 of them in total

for i in range(1,len(filenames)):
    data = pl.read_parquet("../config_02/vectorized_config_02_data_pt%s.parquet"%str(i))
    table.add(data)
    print(f"added vectorized_config_02_data_pt{i}.parquet to database")

added vectorized_config_02_data_pt1.parquet to database
added vectorized_config_02_data_pt2.parquet to database
added vectorized_config_02_data_pt3.parquet to database
added vectorized_config_02_data_pt4.parquet to database
added vectorized_config_02_data_pt5.parquet to database
added vectorized_config_02_data_pt6.parquet to database
added vectorized_config_02_data_pt7.parquet to database
added vectorized_config_02_data_pt8.parquet to database
added vectorized_config_02_data_pt9.parquet to database
added vectorized_config_02_data_pt10.parquet to database
CPU times: user 1min 13s, sys: 15.4 s, total: 1min 28s
Wall time: 1min 45s


### Build ANN Index

In [38]:
%%time
## Build the ANN Index
## https://lancedb.github.io/lancedb/ann_indexes/#creating-an-ivf_pq-index
table.create_index(num_partitions=1024,num_sub_vectors=96,accelerator="mps") ## to use Apple Metal
## It is safe to ignore the UserWarning:...

 52%|██████████████████████▎                    | 26/50 [00:47<00:44,  1.84s/it]


0it [00:00, ?it/s]

CPU times: user 21min 17s, sys: 1min 53s, total: 23min 11s
Wall time: 4min 46s


In [76]:
import time

def make_query(query_text, num_results=20, probes=20, refine_factor=10):
    
    start_time = time.time()
    ## Embed query
    query = model.encode(query_text)

    ## Perform search
    result = table.search(query).limit(num_results).nprobes(20).refine_factor(10).to_polars()
    end_time = time.time()
    print(result.shape)
    result.head()
    result_query_pair = result.with_columns(pl.lit(query_text).alias("query_text"))
    print(f"Time elapsed this query: {(end_time - start_time):.4f} seconds")
    return result_query_pair

In [77]:
data_file = 'Queries.txt'
query_file = open(data_file, 'r')
queries = query_file.readlines()
query_file.close()

# query file is loaded, start asking questions one by one
pl.Config.set_fmt_str_lengths(3000)

# Ask our first query
Replies = make_query(queries[0].strip('\n').strip(' '))
Replies.shape
#print(Replies)
    

(20, 17)
Time elapsed this query: 0.5679 seconds


(20, 18)

In [78]:
# Let's ask the remaining queries

for i in range(1,len(queries)):
    replies = make_query(queries[i].strip('\n').strip(' '))
    
    Replies.extend(replies)

(20, 17)
Time elapsed this query: 0.0917 seconds
(20, 17)
Time elapsed this query: 0.0470 seconds
(20, 17)
Time elapsed this query: 0.0567 seconds
(20, 17)
Time elapsed this query: 0.0524 seconds
(20, 17)
Time elapsed this query: 0.0556 seconds
(20, 17)
Time elapsed this query: 0.0483 seconds
(20, 17)
Time elapsed this query: 0.0528 seconds
(20, 17)
Time elapsed this query: 0.0531 seconds
(20, 17)
Time elapsed this query: 0.0469 seconds
(20, 17)
Time elapsed this query: 0.0572 seconds
(20, 17)
Time elapsed this query: 0.0556 seconds
(20, 17)
Time elapsed this query: 0.0562 seconds
(20, 17)
Time elapsed this query: 0.0536 seconds
(20, 17)
Time elapsed this query: 0.0537 seconds
(20, 17)
Time elapsed this query: 0.0582 seconds
(20, 17)
Time elapsed this query: 0.0511 seconds
(20, 17)
Time elapsed this query: 0.0501 seconds
(20, 17)
Time elapsed this query: 0.0455 seconds
(20, 17)
Time elapsed this query: 0.0472 seconds
(20, 17)
Time elapsed this query: 0.0483 seconds
(20, 17)
Time elapse

In [80]:
# Adding scoring columns for humans

raw_results = Replies.with_columns([pl.lit(None).alias("mo_label"),pl.lit(None).alias("kk_label"),pl.lit(None).alias("kp_label")
                      ,pl.lit(None).alias("dr_label"),pl.lit(None).alias("sr_label")])
print(raw_results.shape)
raw_results.head

#save to parquet file
raw_results.write_parquet('../config_02/query_results_config_02.parquet')

(520, 23)
